### Raw Reddit API Dataframes Exported to PgAdmin

In [5]:
import os
import csv
import psycopg2

# Database connection details
dbname = 'mhdatabase'
user = 'postgres'
password = 'Removed!'
host = 'localhost'
port = '5432'

# Path to the folder containing the CSV files
csv_folder_path = 'raw_api_data'

# List of CSV files
csv_files = [
    'addiction.csv',
    'adhd.csv',
    'anxiety.csv',
    'bipolar.csv',
    'depression.csv',
    'mentalhealth.csv',
    'ocd.csv',
    'ptsd.csv',
    'schizophrenia.csv',
    'selfharm.csv',
    'SuicideWatch.csv'
]

# Data types for each column
data_types = {
    'post_date': 'TIMESTAMP',
    'kind': 'VARCHAR',
    'subreddit': 'VARCHAR',
    'title': 'TEXT',
    'selftext': 'TEXT',
    'username': 'VARCHAR',
    'identifier': 'VARCHAR PRIMARY KEY'
}

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
cursor = conn.cursor()

# Loop through each CSV file
for csv_file in csv_files:
    table_name = csv_file[:-4]  # Remove the '.csv' extension to use as the table name
    file_path = os.path.join(csv_folder_path, csv_file)

    # Import data from the CSV file into the table
    with open(file_path, 'r') as file:
        csv_data = csv.reader(file)
        header = next(csv_data)  # Skip the header row

        # Determine the columns present in this CSV file and create table accordingly
        columns = ', '.join(header)
        column_defs = ', '.join([f'{col} {data_types[col]}' for col in header])

        # Drop the existing table if it exists
        drop_table_query = f"DROP TABLE IF EXISTS {table_name};"
        cursor.execute(drop_table_query)
        conn.commit()

        # Create a new table with the correct column definitions
        create_table_query = f"CREATE TABLE {table_name} ({column_defs});"
        cursor.execute(create_table_query)
        conn.commit()

        placeholders = ', '.join(['%s'] * len(header))
        insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders});"
        cursor.executemany(insert_query, csv_data)
        conn.commit()

# Close the database connection
cursor.close()
conn.close()